<a href="https://colab.research.google.com/github/mouratjim/Instacart_Mouratidis_Mitsas/blob/master/InstacartXKaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd 
import gc                         
gc.enable() 

In [3]:
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
kaggle.json


In [0]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi({"username":"evangeloskalampokis","key":"227307bd0d825ea8f784cb00dc373357"})
api.authenticate()
files = api.competition_download_files("Instacart-Market-Basket-Analysis")

In [0]:
import zipfile
with zipfile.ZipFile('Instacart-Market-Basket-Analysis.zip', 'r') as zip_ref:
    zip_ref.extractall('./input')

In [0]:
import os
working_directory = os.getcwd()+'/input'
os.chdir(working_directory)
for file in os.listdir(working_directory):   
    if zipfile.is_zipfile(file): 
        with zipfile.ZipFile(file) as item: 
           item.extractall()  

In [0]:
orders = pd.read_csv('../input/orders.csv' )
order_products_train = pd.read_csv('../input/order_products__train.csv')
order_products_prior = pd.read_csv('../input/order_products__prior.csv')
products = pd.read_csv('../input/products.csv')
aisles = pd.read_csv('../input/aisles.csv')
departments = pd.read_csv('../input/departments.csv')

In [8]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [9]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [10]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [11]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [12]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [13]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [0]:
aisles['aisle'] = aisles['aisle'].astype('category')
departments['department'] = departments['department'].astype('category')
orders['eval_set'] = orders['eval_set'].astype('category')
products['product_name'] = products['product_name'].astype('category')

In [15]:
op = orders.merge(order_products_prior, on='order_id', how='inner')
op.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [16]:
user = op.groupby('user_id')['order_number'].max().to_frame('user_t_orders') #
user.head()

,user_t_orders
user_id,
1,10
2,14
3,12
4,5
5,4


In [17]:
product_avg_position = op.groupby('product_id')[['add_to_cart_order']].mean()
product_avg_position.columns = ['add_to_cart_mean']
product_avg_position = product_avg_position.reset_index()
product_avg_position.head()

,product_id,add_to_cart_mean
0,1,5.801836
1,2,9.888889
2,3,6.415162
3,4,9.507599
4,5,6.466667


In [18]:
reorprob = op.groupby('product_id').filter(lambda x: x.shape[0] >40)
p_reorprob_results = reorprob.groupby('product_id')['reordered'].mean().to_frame('p_reorder_ratio')
p_reorprob_results = p_reorprob_results.reset_index()
p_reorprob_results.head()


,product_id,p_reorder_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,8,0.503030


In [19]:
up_reorprob_results = reorprob.groupby(['user_id', 'product_id'])['reordered'].mean().to_frame('up_reorder_ratio')
up_reorprob_results = up_reorprob_results.reset_index()
up_reorprob_results.head()


,user_id,product_id,up_reorder_ratio
0,1,196,0.900000
1,1,10258,0.888889
2,1,10326,0.000000
3,1,12427,0.900000
4,1,13032,0.666667


In [20]:
reorprob_results = up_reorprob_results.merge(p_reorprob_results, on='product_id', how='left')
reorprob_results.head()

,user_id,product_id,up_reorder_ratio,p_reorder_ratio
0,1,196,0.900000,0.776480
1,1,10258,0.888889,0.713772
2,1,10326,0.000000,0.652009
3,1,12427,0.900000,0.740735
4,1,13032,0.666667,0.657158


In [21]:
u_reorder_results = op.groupby('user_id')['reordered'].mean().to_frame('u_reordered_ratio')
u_reorder_results = u_reorder_results.reset_index()
u_reorder_results.head()

,user_id,u_reordered_ratio
0,1,0.694915
1,2,0.476923
2,3,0.625000
3,4,0.055556
4,5,0.378378


In [22]:
reorprob_results = reorprob_results.merge(u_reorder_results, on='user_id', how='left')
reorprob_results.head()

,user_id,product_id,up_reorder_ratio,p_reorder_ratio,u_reordered_ratio
0,1,196,0.900000,0.776480,0.694915
1,1,10258,0.888889,0.713772,0.694915
2,1,10326,0.000000,0.652009,0.694915
3,1,12427,0.900000,0.740735,0.694915
4,1,13032,0.666667,0.657158,0.694915


In [23]:
del [reorprob, p_reorprob_results, up_reorprob_results, u_reorder_results]
gc.collect()

44

In [24]:
user = user.reset_index()
user.head()

,user_id,user_t_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [25]:
prd = op.groupby('product_id')['order_id'].count().to_frame('prd_t_purchases') #
prd.head()

,prd_t_purchases
product_id,
1,1852
2,90
3,277
4,329
5,15


In [26]:
prd = prd.reset_index()
prd.head()

,product_id,prd_t_purchases
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [27]:
uxp = op.groupby(['user_id', 'product_id'])['order_id'].count().to_frame('uxp_t_bought') #
uxp.head()


uxp_t_bought
user_id product_id              
1       196                   10
        10258                  9
        10326                  1
        12427                 10
        13032                  3

In [28]:
uxp = uxp.reset_index()
uxp.head()

,user_id,product_id,uxp_t_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [29]:
first_order_no = op.groupby(['user_id', 'product_id'])['order_number'].min().to_frame('first_order_number')
first_order_no  = first_order_no.reset_index()
first_order_no.head()

,user_id,product_id,first_order_number
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [30]:
span = user.merge(first_order_no, on='user_id', how='right')
span['Order_Range_D'] = span.user_t_orders - span.first_order_number + 1
span.head()

,user_id,user_t_orders,product_id,first_order_number,Order_Range_D
0,1,10,196,1,10
1,1,10,10258,2,9
2,1,10,10326,5,6
3,1,10,12427,1,10
4,1,10,13032,2,9


In [31]:
uxp_ratio = uxp.merge(span, on=['user_id', 'product_id'], how='left')
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.uxp_t_bought / uxp_ratio.Order_Range_D
uxp_ratio = uxp_ratio.drop(['uxp_t_bought', 'user_t_orders', 'first_order_number', 'Order_Range_D'], axis=1)
uxp_ratio.head()


,user_id,product_id,uxp_reorder_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.166667
3,1,12427,1.000000
4,1,13032,0.333333


In [32]:
del [first_order_no, span]
gc.collect()

77

In [33]:
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'])
uxp.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio
0,1,196,10,1.000000
1,1,10258,9,1.000000
2,1,10326,1,0.166667
3,1,12427,10,1.000000
4,1,13032,3,0.333333


In [34]:
del uxp_ratio
gc.collect()

11

In [35]:
op['order_number_back'] = op.groupby('user_id')['order_number'].transform(max) - op.order_number +1 
op.head(15)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,2539329,1,prior,1,2,8,NaN,196,1,0,10
1,2539329,1,prior,1,2,8,NaN,14084,2,0,10
2,2539329,1,prior,1,2,8,NaN,12427,3,0,10
3,2539329,1,prior,1,2,8,NaN,26088,4,0,10
4,2539329,1,prior,1,2,8,NaN,26405,5,0,10
5,2398795,1,prior,2,3,7,15.0,196,1,1,9
6,2398795,1,prior,2,3,7,15.0,10258,2,0,9
7,2398795,1,prior,2,3,7,15.0,12427,3,1,9
8,2398795,1,prior,2,3,7,15.0,13176,4,0,9
9,2398795,1,prior,2,3,7,15.0,26088,5,1,9


In [36]:
op5 = op[op.order_number_back <= 5]
op5.head(15)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
29,3367565,1,prior,6,2,7,19.0,196,1,1,5
30,3367565,1,prior,6,2,7,19.0,12427,2,1,5
31,3367565,1,prior,6,2,7,19.0,10258,3,1,5
32,3367565,1,prior,6,2,7,19.0,25133,4,1,5
33,550135,1,prior,7,1,9,20.0,196,1,1,4
34,550135,1,prior,7,1,9,20.0,10258,2,1,4
35,550135,1,prior,7,1,9,20.0,12427,3,1,4
36,550135,1,prior,7,1,9,20.0,25133,4,1,4
37,550135,1,prior,7,1,9,20.0,13032,5,1,4
38,3108588,1,prior,8,1,14,14.0,12427,1,1,3


In [37]:
del op
gc.collect()

22

In [38]:
last_five = op5.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']
last_five.head(10)

times_last5
user_id product_id             
1       196                   5
        10258                 5
        12427                 5
        13032                 2
        25133                 5
        35951                 1
        38928                 1
        39657                 1
        46149                 3
        49235                 2

In [39]:
uxp = uxp.merge(last_five, on=['user_id', 'product_id'], how='left')
uxp.head(15)

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5
0,1,196,10,1.000000,5.0
1,1,10258,9,1.000000,5.0
2,1,10326,1,0.166667,NaN
3,1,12427,10,1.000000,5.0
4,1,13032,3,0.333333,2.0
5,1,13176,2,0.222222,NaN
6,1,14084,1,0.100000,NaN
7,1,17122,1,0.166667,NaN
8,1,25133,8,1.000000,5.0
9,1,26088,2,0.200000,NaN


In [40]:
del [op5, last_five]
gc.collect()

22

In [41]:
data = uxp.merge(user, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders
0,1,196,10,1.000000,5.0,10
1,1,10258,9,1.000000,5.0,10
2,1,10326,1,0.166667,NaN,10
3,1,12427,10,1.000000,5.0,10
4,1,13032,3,0.333333,2.0,10


In [42]:
data = data.merge(prd, on='product_id', how='left') #
data.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,prd_t_purchases
0,1,196,10,1.000000,5.0,10,35791
1,1,10258,9,1.000000,5.0,10,1946
2,1,10326,1,0.166667,NaN,10,5526
3,1,12427,10,1.000000,5.0,10,6476
4,1,13032,3,0.333333,2.0,10,3751


In [43]:
data = data.merge(product_avg_position, on='product_id', how='left')
data.head()


,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,prd_t_purchases,add_to_cart_mean
0,1,196,10,1.000000,5.0,10,35791,3.721774
1,1,10258,9,1.000000,5.0,10,1946,4.277492
2,1,10326,1,0.166667,NaN,10,5526,4.191097
3,1,12427,10,1.000000,5.0,10,6476,4.760037
4,1,13032,3,0.333333,2.0,10,3751,5.622767


In [44]:
del [uxp, user, prd, product_avg_position]
gc.collect()


0

In [45]:
data = data.merge(reorprob_results, on=['user_id', 'product_id'], how='left')
data.head()


,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,prd_t_purchases,add_to_cart_mean,up_reorder_ratio,p_reorder_ratio,u_reordered_ratio
0,1,196,10,1.000000,5.0,10,35791,3.721774,0.900000,0.776480,0.694915
1,1,10258,9,1.000000,5.0,10,1946,4.277492,0.888889,0.713772,0.694915
2,1,10326,1,0.166667,NaN,10,5526,4.191097,0.000000,0.652009,0.694915
3,1,12427,10,1.000000,5.0,10,6476,4.760037,0.900000,0.740735,0.694915
4,1,13032,3,0.333333,2.0,10,3751,5.622767,0.666667,0.657158,0.694915


In [46]:
del reorprob_results
gc.collect()

11

In [47]:
orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]
orders_future = orders_future[ ['user_id', 'eval_set', 'order_id'] ]
orders_future.head(10)


,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797
53,6,test,1528013
74,7,train,525192
78,8,train,880375
82,9,train,1094988
88,10,train,1822501


In [48]:
data = data.merge(orders_future, on='user_id', how='left')
data.head(10)

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,prd_t_purchases,add_to_cart_mean,up_reorder_ratio,p_reorder_ratio,u_reordered_ratio,eval_set,order_id
0,1,196,10,1.000000,5.0,10,35791,3.721774,0.900000,0.776480,0.694915,train,1187899
1,1,10258,9,1.000000,5.0,10,1946,4.277492,0.888889,0.713772,0.694915,train,1187899
2,1,10326,1,0.166667,NaN,10,5526,4.191097,0.000000,0.652009,0.694915,train,1187899
3,1,12427,10,1.000000,5.0,10,6476,4.760037,0.900000,0.740735,0.694915,train,1187899
4,1,13032,3,0.333333,2.0,10,3751,5.622767,0.666667,0.657158,0.694915,train,1187899
5,1,13176,2,0.222222,NaN,10,379450,5.095947,0.500000,0.832555,0.694915,train,1187899
6,1,14084,1,0.100000,NaN,10,15935,5.792595,0.000000,0.810982,0.694915,train,1187899
7,1,17122,1,0.166667,NaN,10,13880,6.257421,0.000000,0.675576,0.694915,train,1187899
8,1,25133,8,1.000000,5.0,10,6196,7.001614,0.875000,0.740155,0.694915,train,1187899
9,1,26088,2,0.200000,NaN,10,2523,6.495838,0.500000,0.539041,0.694915,train,1187899


In [49]:
del orders_future
gc.collect()

22

In [50]:
data_train = data[data.eval_set=='train'] #
data_train.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,prd_t_purchases,add_to_cart_mean,up_reorder_ratio,p_reorder_ratio,u_reordered_ratio,eval_set,order_id
0,1,196,10,1.000000,5.0,10,35791,3.721774,0.900000,0.776480,0.694915,train,1187899
1,1,10258,9,1.000000,5.0,10,1946,4.277492,0.888889,0.713772,0.694915,train,1187899
2,1,10326,1,0.166667,NaN,10,5526,4.191097,0.000000,0.652009,0.694915,train,1187899
3,1,12427,10,1.000000,5.0,10,6476,4.760037,0.900000,0.740735,0.694915,train,1187899
4,1,13032,3,0.333333,2.0,10,3751,5.622767,0.666667,0.657158,0.694915,train,1187899


In [51]:
data_train = data_train.merge(order_products_train[['product_id','order_id', 'reordered']], on=['product_id','order_id'], how='left' )
data_train.head(15)

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,prd_t_purchases,add_to_cart_mean,up_reorder_ratio,p_reorder_ratio,u_reordered_ratio,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,10,35791,3.721774,0.900000,0.776480,0.694915,train,1187899,1.0
1,1,10258,9,1.000000,5.0,10,1946,4.277492,0.888889,0.713772,0.694915,train,1187899,1.0
2,1,10326,1,0.166667,NaN,10,5526,4.191097,0.000000,0.652009,0.694915,train,1187899,NaN
3,1,12427,10,1.000000,5.0,10,6476,4.760037,0.900000,0.740735,0.694915,train,1187899,NaN
4,1,13032,3,0.333333,2.0,10,3751,5.622767,0.666667,0.657158,0.694915,train,1187899,1.0
5,1,13176,2,0.222222,NaN,10,379450,5.095947,0.500000,0.832555,0.694915,train,1187899,NaN
6,1,14084,1,0.100000,NaN,10,15935,5.792595,0.000000,0.810982,0.694915,train,1187899,NaN
7,1,17122,1,0.166667,NaN,10,13880,6.257421,0.000000,0.675576,0.694915,train,1187899,NaN
8,1,25133,8,1.000000,5.0,10,6196,7.001614,0.875000,0.740155,0.694915,train,1187899,1.0
9,1,26088,2,0.200000,NaN,10,2523,6.495838,0.500000,0.539041,0.694915,train,1187899,1.0


In [52]:
data_train['times_last5'] = data_train['times_last5'].fillna(0)
data_train['p_reorder_ratio'] = data_train['p_reorder_ratio'].fillna(0)
data_train['up_reorder_ratio'] = data_train['up_reorder_ratio'].fillna(0)
data_train['u_reordered_ratio'] = data_train['u_reordered_ratio'].fillna(0)
data_train['reordered'] = data_train['reordered'].fillna(0)
data_train.head(15)

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,prd_t_purchases,add_to_cart_mean,up_reorder_ratio,p_reorder_ratio,u_reordered_ratio,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,10,35791,3.721774,0.900000,0.776480,0.694915,train,1187899,1.0
1,1,10258,9,1.000000,5.0,10,1946,4.277492,0.888889,0.713772,0.694915,train,1187899,1.0
2,1,10326,1,0.166667,0.0,10,5526,4.191097,0.000000,0.652009,0.694915,train,1187899,0.0
3,1,12427,10,1.000000,5.0,10,6476,4.760037,0.900000,0.740735,0.694915,train,1187899,0.0
4,1,13032,3,0.333333,2.0,10,3751,5.622767,0.666667,0.657158,0.694915,train,1187899,1.0
5,1,13176,2,0.222222,0.0,10,379450,5.095947,0.500000,0.832555,0.694915,train,1187899,0.0
6,1,14084,1,0.100000,0.0,10,15935,5.792595,0.000000,0.810982,0.694915,train,1187899,0.0
7,1,17122,1,0.166667,0.0,10,13880,6.257421,0.000000,0.675576,0.694915,train,1187899,0.0
8,1,25133,8,1.000000,5.0,10,6196,7.001614,0.875000,0.740155,0.694915,train,1187899,1.0
9,1,26088,2,0.200000,0.0,10,2523,6.495838,0.500000,0.539041,0.694915,train,1187899,1.0


In [53]:
data_train = data_train.set_index(['user_id', 'product_id'])
data_train.head(15)

uxp_t_bought  uxp_reorder_ratio  ...  order_id  reordered
user_id product_id                                   ...                     
1       196                   10           1.000000  ...   1187899        1.0
        10258                  9           1.000000  ...   1187899        1.0
        10326                  1           0.166667  ...   1187899        0.0
        12427                 10           1.000000  ...   1187899        0.0
        13032                  3           0.333333  ...   1187899        1.0
        13176                  2           0.222222  ...   1187899        0.0
        14084                  1           0.100000  ...   1187899        0.0
        17122                  1           0.166667  ...   1187899        0.0
        25133                  8           1.000000  ...   1187899        1.0
        26088                  2           0.200000  ...   1187899        1.0
        26405                  2           0.200000  ...   1187899        1.0
        30450                  1           0.125000  ...   1187899        0.0
        35951                  1           1.000000  ...   1187899        0.0
        38928                  1           1.000000  ...   1187899        1.0
        39657                  1           1.000000  ...   1187899        1.0

[15 rows x 12 columns]

In [54]:
data_train = data_train.drop(['eval_set', 'order_id'], axis=1)
data_train.head(15)

uxp_t_bought  ...  reordered
user_id product_id                ...           
1       196                   10  ...        1.0
        10258                  9  ...        1.0
        10326                  1  ...        0.0
        12427                 10  ...        0.0
        13032                  3  ...        1.0
        13176                  2  ...        0.0
        14084                  1  ...        0.0
        17122                  1  ...        0.0
        25133                  8  ...        1.0
        26088                  2  ...        1.0
        26405                  2  ...        1.0
        30450                  1  ...        0.0
        35951                  1  ...        0.0
        38928                  1  ...        1.0
        39657                  1  ...        1.0

[15 rows x 10 columns]

In [55]:
data_test = data[data.eval_set=='test'] #
data_test.head()

,user_id,product_id,uxp_t_bought,uxp_reorder_ratio,times_last5,user_t_orders,prd_t_purchases,add_to_cart_mean,up_reorder_ratio,p_reorder_ratio,u_reordered_ratio,eval_set,order_id
120,3,248,1,0.090909,NaN,12,6371,10.620782,0.000000,0.400251,0.625,test,2774568
121,3,1005,1,0.333333,1.0,12,463,9.498920,0.000000,0.440605,0.625,test,2774568
122,3,1819,3,0.333333,NaN,12,2424,9.287541,0.666667,0.492162,0.625,test,2774568
123,3,7503,1,0.100000,NaN,12,12474,9.547379,0.000000,0.553551,0.625,test,2774568
124,3,8021,1,0.090909,NaN,12,27864,8.822854,0.000000,0.591157,0.625,test,2774568


In [56]:
data_test = data_test.set_index(['user_id', 'product_id']) 
data_test.head()

uxp_t_bought  uxp_reorder_ratio  ...  eval_set  order_id
user_id product_id                                   ...                    
3       248                    1           0.090909  ...      test   2774568
        1005                   1           0.333333  ...      test   2774568
        1819                   3           0.333333  ...      test   2774568
        7503                   1           0.100000  ...      test   2774568
        8021                   1           0.090909  ...      test   2774568

[5 rows x 11 columns]

In [57]:
data_test = data_test.drop(['eval_set','order_id'], axis=1)
data_test.head()

uxp_t_bought  ...  u_reordered_ratio
user_id product_id                ...                   
3       248                    1  ...              0.625
        1005                   1  ...              0.625
        1819                   3  ...              0.625
        7503                   1  ...              0.625
        8021                   1  ...              0.625

[5 rows x 9 columns]

In [58]:
data_test['times_last5'] = data_test['times_last5'].fillna(0)
data_test['p_reorder_ratio'] = data_test['p_reorder_ratio'].fillna(0)
data_test['up_reorder_ratio'] = data_test['up_reorder_ratio'].fillna(0)
data_test['u_reordered_ratio'] = data_test['u_reordered_ratio'].fillna(0)
data_test.head()

uxp_t_bought  ...  u_reordered_ratio
user_id product_id                ...                   
3       248                    1  ...              0.625
        1005                   1  ...              0.625
        1819                   3  ...              0.625
        7503                   1  ...              0.625
        8021                   1  ...              0.625

[5 rows x 9 columns]

In [0]:
from sklearn.ensemble import RandomForestClassifier
X_train, y_train = data_train.drop('reordered', axis=1), data_train.reordered
rfc = RandomForestClassifier(n_estimators=20,max_depth=6, n_jobs=-1 ,random_state=50)
model = rfc.fit(X_train, y_train)

feature_importances_df = pd.DataFrame(model.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances_df)
feat_importances = pd.Series(model.feature_importances_, index=X_train.columns).sort_values()
feat_importances.plot(kind='barh')

del[X_train, y_train]
gc.collect

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier

X_train, y_train = data_train.drop('reordered', axis=1), data_train.reordered
lr = LogisticRegression(random_state=42, solver='lbfgs')
bagging_lr = BaggingClassifier(base_estimator=lr, n_estimators=10)
model = bagging_lr.fit(X_train, y_train) 

del[X_train, y_train]
gc.collect

In [0]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
import xgboost as xgb 
X_train, y_train = data_train.drop('reordered', axis=1), data_train.reordered
rc = RidgeClassifier()
parameters = {'eval_metric':'logloss', 'max_depth':'5', 'colsample_bytree':'0.4', 'subsample':'0.75'}
xgbc = xgb.XGBClassifier(objective='binary:logistic', parameters=parameters, num_boost_round=10)
classifiers1 = [('Logistic Regression', bagging_lr), ('Random Forest Tree', rfc), ('XGBClassifier', xgbc)]
classifiers2 = [('Logistic Regression', bagging_lr), ('Random Forest Tree', rfc), ('XGBClassifier', xgbc), ('RidgeClassifier', rc)]
vc1 = VotingClassifier(estimators=classifiers1, voting='soft')
vc2 = VotingClassifier(estimators=classifiers2)
model1 = vc1.fit(X_train, y_train)
model2 = vc2.fit(X_train, y_train)
del [X_train, y_train]
gc.collect()

In [0]:
test_pred = model2.predict(data_test).astype(int)
test_pred[0:20]

In [0]:
test_pred = (model1.predict_proba(data_test)[:,1] >= 0.21).astype(int)
test_pred[0:20]

In [0]:
data_test['prediction'] = test_pred
data_test.head(20)

In [0]:
final = data_test.reset_index()
final = final[['product_id', 'user_id', 'prediction']]

gc.collect()
final.head()

In [0]:
orders_test = orders.loc[orders.eval_set=='test',("user_id", "order_id") ]
orders_test.head()

In [0]:
final = final.merge(orders_test, on='user_id', how='left')
final.head()

In [0]:
#remove user_id column
final = final.drop('user_id', axis=1)
#convert product_id as integer
final['product_id'] = final.product_id.astype(int)

#Remove all unnecessary objects
del orders
del orders_test
gc.collect()

final.head()

In [0]:
d = dict()
for row in final.itertuples():
    if row.prediction== 1:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

for order in final.order_id:
    if order not in d:
        d[order] = 'None'
        
gc.collect()

In [0]:
#Convert the dictionary into a DataFrame
sub = pd.DataFrame.from_dict(d, orient='index')

#Reset index
sub.reset_index(inplace=True)
#Set column names
sub.columns = ['order_id', 'products']

sub.head()

In [0]:
sub.shape[0]

In [0]:
sub.to_csv('sub.csv', index=False)